In [1]:
import numpy as np
import pandas as pd
import re

from IPython.display import display
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize  
from collections import Counter
import string

from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.utils import pprint
from konlpy.tag import Twitter

import os

from tqdm import tqdm
import ast

from gensim.models import Word2Vec
import random

In [2]:
kkma = Kkma()

In [3]:
file_list = os.listdir(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data")
file_list_xlsx = [f for f in file_list if f.endswith(".xlsx")]
file_naver = [f for f in file_list_xlsx if f.endswith("네이버.xlsx")]
file_insta = [f for f in file_list_xlsx if f.endswith("인스타그램.xlsx")]

### dic 에 있는 file list

In [4]:
version = "네이버"

def file_extraction(f_list,version):
    pattern = r"\d* ([\w|\s]*)_{}.xlsx".format(version)
    #print(pattern)
    reg = re.compile(pattern)
    list_ = []
    for file in f_list:
        search = reg.search(file)
        try:
            target = search.group(1)
            list_.append(target)
        except:
            pass
    return list_

### data 읽기

In [24]:
taget_ = '맨 케어'
path_ = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\{}"

def loading_data(target_,version,file_list,path_):
    pat = r".*{}_{}".format(target_,version)
    reg_2 = re.compile(pat)
    file_ = list(filter(reg_2.match,file_list))[0]
    data_ = pd.ExcelFile(path_.format(file_))
    return data_

### concat data sheet

In [6]:
def concat_sheet(excel):
    sheet_list = excel.sheet_names
    df_ = pd.DataFrame()
    for sheet in sheet_list:
        sheet_data = excel.parse(sheet,encoding='cp949' )
        sheet_data["Sheet"] = sheet
        df_ = pd.concat([df_,sheet_data]).reset_index(drop=True)
    if "naver_blog_review" in df_.columns:
        df_ = df_.rename(columns={"naver_blog_review":"review"})
        df_["From"] = "Naver"
    else:
        df_ = df_.rename(columns={"insta_review":"review"})
        df_["From"] = "Insta"
    return df_


### concat naver & insta

In [7]:
def concat_na_ins(target_,file_list):
    ver_ = "네이버"
    ta = target_
    
    ver_ = "네이버"
    naver_ = loading_data(ta,ver_,file_list)
    a = concat_sheet(naver_)

    ver_ = "인스타그램"
    insta_ = loading_data(ta,ver_,file_list)
    b = concat_sheet(insta_)
    df_naver = a.loc[:,["제품명(영문)","review","Sheet","From"]]
    df_insta = b.loc[:,["제품명(영문)","review","Sheet","From"]]
    df_all = pd.concat([df_naver,df_insta]).reset_index(drop=True)
    return df_all

In [14]:
ver_ = "네이버"
ta = file_extraction(file_list,ver_)[1]
ta

'맨 케어'

In [15]:
df_a = concat_na_ins(ta,file_list)

### tagging

In [8]:
def pos_tagging(df_):
    pos = []
    words = []
    #sentences = []
    for i in tqdm(range(len(df_))):
        try:
            p = kkma.pos(df_["review"][i])
            #senten = kkma.sentences(df_["review"][i])
        except:
            text = df_["review"][i] + "']"
            #senten = []
            try:
                p = kkma.pos(text)
            except:
                p = []
        word = [w[0] for w in p]
        pos.append(p)
        words.append(word)
        #sentences.append(senten)
    df_["tag"] = pos
    df_["words"] = words
    #df_["sentences"] = sentences
    return df_

In [9]:
def pos_tagging_long(df_):
    pos = []
    words = []
    sentences = []
    for i in tqdm(range(len(df_))):
        try:
            p = kkma.pos(df_["review"][i])
            senten = kkma.sentences(df_["review"][i])
        except:
            text = df_["review"][i] + "']"
            senten = []
            try:
                p = kkma.pos(text)
            except:
                p = []
        word = [w[0] for w in p]
        pos.append(p)
        words.append(word)
        sentences.append(senten)
    df_["tag"] = pos
    df_["words"] = words
    df_["sentences"] = sentences
    return df_

In [ ]:
df_test = df_a[0:10]
df_tag = pos_tagging(df_test)

### tag별 나누기

In [11]:
#df = df_tag.copy()
tag_dic = {
    "Noun":["NNG","NNP","NNM"],
    "Verb":['VV', 'VA', 'VXV', 'VCN'],
    "Adverb":["MAG","MAC"],
    "Adj":["XPV","XSM","XSO","XR"],
    "Unknown":['UN', 'UV', 'UE'],
    "ETC":["EMO","OL"],
    "All" : ["NNG","NNP","NNM",'VV', 'VA', 'VXV', 'VCN',"MAG","MAC","XPV","XSM","XSO","XR",'UN', 'UV', 'UE',"EMO","OL"]
}



def tag_split(df,tag_dic):
    def only_(df,igs):
        list_1 = []
        for list_ in df.loc[:,"tag"]:
            list_2 = []
            for i in range(len(list_)):
                #print(list_[i])
                if list_[i][1] in igs:
                    list_2.append(list_[i][0])
            list_1.append(list_2)
        return list_1

    def ignore_(df,igs):
        list_1 = []
        for list_ in df.loc[:,"tag"]:
            list_2 = list_.copy()
            for i in range(len(list_)):
                #print(list_[i])
                if list_[i][1] in igs:
                    list_2.remove(list_[i])
            list_3 = [w[0] for w in list_2]
            list_1.append(list_3)
        return list_1
    df.loc[:,"Noun"] = only_(df,tag_dic["Noun"])
    df.loc[:,"Verb"] = only_(df,tag_dic["Verb"])
    df.loc[:,"Adverb"] = only_(df,tag_dic["Adverb"])
    df.loc[:,"Adj"] = only_(df,tag_dic["Adj"])
    df.loc[:,"Unknown"] = only_(df,tag_dic["Unknown"])
    df.loc[:,"ETC"] = only_(df,tag_dic["ETC"])
    df.loc[:,"All"] = only_(df,tag_dic["All"])
    return df

In [ ]:
df_split = tag_split(df_tag,tag_dic)
df_split

### word2vec 학습기 ( 나누기 전에)

In [12]:
#df_all = df_split
win_size = 5
target_col = "All"

def word2vec_model(df_all,target_col,win_size):
    model_words = Word2Vec(sentences=df_all[target_col], window = win_size, min_count=0,iter=100, sg=1)
    return model_words

In [ ]:
model_word = word2vec_model(df_split,"All",5)

### make_most df

In [16]:
target_list = list(tag_dic.keys())
target_tag = target_list[0]
target_tag

'Noun'

In [17]:
most_n = 30
#df_ = df_split

def make_mostdf(df_,target_tag,most_n):
    df_most = pd.DataFrame()
    word = Counter(df_[target_tag].sum()).most_common(most_n)
    df_most["word"] = [n[0] for n in word]
    df_most["count"] = [n[1] for n in word]
    return df_most


In [ ]:
dm = make_mostdf(df_split,target_tag,40)

### 해당 문장 찾기

In [210]:
df_ = df_split
df_m = dm.copy()

In [214]:
df_.head()

,제품명(영문),review,Sheet,From,tag,words,sentences,Noun,Verb,Adverb,Adj,Unknown,ETC,All
0,Super Moisture Gel,[],01 기초 스킨 케어,Naver,"[([, SS), (], SS)]","[[, ]]",[[]],[],[],[],[],[],[],[]
1,SIGNATURE ALL IN ONE ESSENCE MOISTURE PLUS,['끈적임없는 남자에센스추천 포맨트 시그니처 올인원 모이스처 플러스 ...,01 기초 스킨 케어,Naver,"[([, SS), (', SS), (끈적이, VV), (ㅁ, ETN), (없, VA...","[[, ', 끈적이, ㅁ, 없, 는, 남자, 에, 센스, 추천, 포, 맨트, 시, ...",[[' 끈적임 없는 남자에 센스 추천 포 맨트 시 그니 처 올인원 모이 스처 플러스...,"[남자, 센스, 추천, 포, 시, 처, 올인원, 모이, 플러스, 안녕, 뷰, 스타,...","[끈적이, 없, 그, 아니, 갖, 블, 그, 두, 그리하, 되, 하, 바르, 바쁘,...","[안, 늘, 또, 안, 다, 없이, 다, 특히나, 더, 많이, 잘, 너무나, 꽤, ...","[민감, 촉촉, 산뜻, 답답, 갑갑, 촉촉, 깔끔, 촉촉, 답답, 흐뭇, 산뜻, 산...","[맨트, 스처, 린, 트시, 스처, 있고보습과, 맨트, 스처, 준비해야겠더라고, 맨...","[ㅋㅋ, ml, ㅎㅎ, ^^;, ㅋㅋ, ㅠ_ㅠ, ㅋㅋ, ^^;;, ml, ㅋㅋ, ^...","[끈적이, 없, 남자, 센스, 추천, 포, 맨트, 시, 그, 처, 올인원, 모이, ..."
2,SIGNATURE ALL IN ONE ESSENCE,['끈적임없는 남자에센스추천 포맨트 시그니처 올인원 모이스처 플러스 ...,01 기초 스킨 케어,Naver,"[([, SS), (', SS), (끈적이, VV), (ㅁ, ETN), (없, VA...","[[, ', 끈적이, ㅁ, 없, 는, 남자, 에, 센스, 추천, 포, 맨트, 시, ...",[[' 끈적임 없는 남자에 센스 추천 포 맨트 시 그니 처 올인원 모이 스처 플러스...,"[남자, 센스, 추천, 포, 시, 처, 올인원, 모이, 플러스, 안녕, 뷰, 스타,...","[끈적이, 없, 그, 아니, 갖, 블, 그, 두, 그리하, 되, 하, 바르, 바쁘,...","[안, 늘, 또, 안, 다, 없이, 다, 특히나, 더, 많이, 잘, 너무나, 꽤, ...","[민감, 촉촉, 산뜻, 답답, 갑갑, 촉촉, 깔끔, 촉촉, 답답, 간단, 산뜻, 어...","[맨트, 스처, 린, 트시, 스처, 있고보습과, 맨트, 스처, 맨트, 스처, 스처,...","[ㅋㅋ, ml, ㅎㅎ, ml, ㅎㅎ, ml, ml, ^^, ㅠㅠ, ㅠㅠ, ㅠㅠ, ㅠ...","[끈적이, 없, 남자, 센스, 추천, 포, 맨트, 시, 그, 처, 올인원, 모이, ..."
3,AQUAPOWER CLEAR ESSENCE,['남자스킨 써보니 역시! 아쿠아파워 올인원 비오템 옴므 안녕하세요!...,01 기초 스킨 케어,Naver,"[([, SS), (', SS), (남자, NNG), (스킨, NNG), (쓰, V...","[[, ', 남자, 스킨, 쓰, 어, 보, 니, 역시, !, 아쿠아, 파워, 올인원...","[[' 남자 스킨 써 보니 역시! 아쿠아 파워 올인원 비오 템 옴 므 안녕하세요!,...","[남자, 스킨, 아쿠아, 파워, 올인원, 비, 오, 옴, 안녕, 패션, 뷰티, 로,...","[쓰, 보, 블, 그, 빠르, 다가오, 느껴지, 춥, 쉽, 쉽, 던지, 하, 아니,...","[역시, 부쩍, 이렇게, 가장, 특히나, 특히, 물론, 아무래도, 조금, 더, 가장...","[강력, 자세, 궁금, 풍부, 촉촉, 은은, 촉촉, 간편, 촉촉, 풍부, 이만, 풍...","[므, 피부, 그루밍제품, 그루밍, 옴므라, 틱, 틱, 컨셉, 비오템옴므, 쥰, 비...","[BIOTHERM, HOMME, Needs, T-PUR, T-PUR, BIOTHER...","[남자, 스킨, 쓰, 보, 역시, 아쿠아, 파워, 올인원, 비, 오, 옴, 므, 안..."
4,FORCE SUPREME DUAL AMPOULE,['여자들만큼 남자들도 피부에 따라서 나이가 달라보이고 전체적인 느낌이 달라지잖아요...,01 기초 스킨 케어,Naver,"[([, SS), (', SS), (여자, NNG), (들, XSN), (만큼, N...","[[, ', 여자, 들, 만큼, 남자, 들, 도, 피부, 에, 따르, 아서, 나이,...",[[' 여자들 만큼 남자들도 피부에 따라서 나이가 달라 보이고 전체적인 느낌이 달라...,"[여자, 만큼, 남자, 피부, 나이, 전체적, 느낌, 만큼, 오빠, 동생, 애인, ...","[따르, 닿, 보이, 달라지, 그러, 있, 있, 드리, 저, 잦, 피곤하, 보이, ...","[마침, 점점, 따로, 너무, 바로, 정말, 많이, 잘, 잘, 그리고, 더, 지금,...","[탄탄, 칙칙, 강력, 탄탄, 강력, 강력, 강렬, 시원, 시원, 산뜻, 산뜻, 푸...","[앰플, 므, 앰플, 프트, 므, 앰플, 이스, 기프트박스, 므, 앰플, 닝, 앰플...","[ml, ml, ml, ml, ^^, C, C, ㅎㅎ, ㅎㅎ, CLICK, ^^, ...","[여자, 만큼, 남자, 피부, 따르, 나이, 닿, 보이, 전체적, 느낌, 달라지, ..."


In [19]:
def find_sentence_long(df_,df_m):
    exts = []
    for tar in df_m["word"]:
        target = tar
        extraction = []
        for sentences in df_["sentences"]:
            for senten in sentences:
                if target in senten:
                    extraction.append(senten)

        exts.append(extraction)
    df_m["setence"] = exts
    return df_m

In [20]:
window_size = 10
def find_sentence_short(df_,df_m,window_size):
    exts = []
    for tar in df_m["word"]:
        target = tar
        extraction = []
        for i in df_["review"]:
            sentences = i.split("  ")
            for senten in sentences:
                if target in senten:
                    index_ = senten.find(target)
                    #extraction.append(senten)
                    try:
                        extraction.append(senten[index_-window_size:index_+window_size])
                    except:
                        pass
        exts.append(extraction)
    df_m["setence"] = exts
    return df_m

In [ ]:
df_ = df_split
df_m = dm.copy()

df_m = find_sentence_short(df_split,dm,10)

### word2vec similar 넣기

In [22]:
def most_similar(df_,w2_model):
    most_simi = []
    for i in df_["word"]:
        ms = w2_model.wv.most_similar(i,topn=20)
        most_simi.append(ms)
    df_["word2vec"] = most_simi
    return df_

In [ ]:
#df_ = dm
w2_model = model_word

df_m = most_similar(dm,model_word)
df_m.head()

In [ ]:
ver_ = "네이버"
ta = file_extraction(file_list,ver_)[1]
ta

In [ ]:
df_a = concat_na_ins(ta,file_list)

In [ ]:
df_test = df_a[0:10]
df_tag = pos_tagging(df_test)

In [ ]:
df_split = tag_split(df_tag,tag_dic)
df_split

In [ ]:
model_word = word2vec_model(df_split,"All",5)

In [ ]:
target_list = list(tag_dic.keys())
target_tag = target_list[0]
target_tag

In [ ]:

dm = make_mostdf(df_split,target_tag,40)

In [ ]:
df_ = df_split
df_m = dm.copy()

df_m = find_sentence_short(df_split,dm,10)

In [ ]:
#df_ = dm
w2_model = model_word

df_m = most_similar(dm,model_word)
df_m.head()